### Quantopian Pipelines Filters

In [1]:
from quantopian.pipeline import Pipeline

In [2]:
def make_pipeline():
    return Pipeline()

In [3]:
pipe = make_pipeline()

In [4]:
from quantopian.research import run_pipeline

In [5]:
result = run_pipeline(pipe,'2017-01-03','2020-01-06')

In [6]:
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import SimpleMovingAverage

In [7]:
SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=30)

SimpleMovingAverage([EquityPricing<US>.close], 30)

In [8]:
def make_pipeline():
    
    latest_close = USEquityPricing.close.latest
    small_price = latest_close < 5
    
    mean_close_30 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=30,mask=small_price)
    mean_close_10 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=10,mask=small_price)
    
    percent_difference = (mean_close_10 - mean_close_30) / mean_close_30
    
    perc_filter = percent_difference > 0
    
    # COMBINE FILTERS
    final_filter = perc_filter & small_price
    
    return Pipeline(columns={'30 Day Mean Close':mean_close_30,
                             'Percent Difference':percent_difference,
                            'Latest Close':latest_close,
                            'Percent Filter':perc_filter
                            },screen=final_filter)

In [9]:
results = run_pipeline(make_pipeline(),'2020-07-06','2020-07-06')
results.head()

30 Day Mean Close  Latest Close  \
2020-07-06 00:00:00+00:00 Equity(117 [AEY])            2.645667         3.220   
                          Equity(157 [AEG])            2.946100         3.010   
                          Equity(392 [AMS])            2.074433         2.110   
                          Equity(535 [ARTW])           2.290200         2.460   
                          Equity(680 [AXR])            4.492038         4.682   

                                              Percent Difference  \
2020-07-06 00:00:00+00:00 Equity(117 [AEY])             0.082147   
                          Equity(157 [AEG])             0.000407   
                          Equity(392 [AMS])             0.057831   
                          Equity(535 [ARTW])            0.057899   
                          Equity(680 [AXR])             0.009069   

                                             Percent Filter  
2020-07-06 00:00:00+00:00 Equity(117 [AEY])            True  
                          Equity(157 [AEG])            True  
                          Equity(392 [AMS])            True  
                          Equity(535 [ARTW])           True  
                          Equity(680 [AXR])            True

In [10]:
results.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 718 entries, (2020-07-06 00:00:00+00:00, Equity(117 [AEY])) to (2020-07-06 00:00:00+00:00, Equity(54258 [IPOC_WS]))
Data columns (total 4 columns):
30 Day Mean Close     718 non-null float64
Latest Close          718 non-null float64
Percent Difference    718 non-null float64
Percent Filter        718 non-null bool
dtypes: bool(1), float64(3)
memory usage: 23.1+ KB


### Quantopian Pipelines Masking and Classifiers
Masking allows us to tell pipeline to ignore assets all together, before the factors or filters even really take place

We can pass in the mask parameter to both factors and filters

Classifiers take in an asset and a timestamp and return a categorical value, such as a sector or an exchange

In [11]:
from quantopian.pipeline.data import morningstar
from quantopian.pipeline.classifiers.morningstar import Sector

In [12]:
morningstart_sector = Sector()

In [13]:
exchange = morningstar.share_class_reference.exchange_id.latest

In [14]:
nyse_filter = exchange.eq('NYS')

In [15]:
def make_pipeline():
    
    latest_close = USEquityPricing.close.latest
    small_price = latest_close < 5
    
    # CLASSIFIER
    nyse_filter = exchange.eq('NYS')
    
    mean_close_30 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=30,mask=small_price)
    mean_close_10 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=10,mask=small_price)
    
    percent_difference = (mean_close_10 - mean_close_30) / mean_close_30
    
    perc_diff_check = percent_difference > 0
    
    # COMBINE FILTERS
    final_filter = perc_diff_check & nyse_filter
    
    return Pipeline(columns={'30 Day Mean Close':mean_close_30,
                             'Percent Difference':percent_difference,
                            'Latest Close':latest_close,
                            'Percent Filter':perc_diff_check
                            },screen=final_filter)

In [16]:
results = run_pipeline(make_pipeline(),'2020-07-06','2020-07-06')
results.head()

30 Day Mean Close  Latest Close  \
2020-07-06 00:00:00+00:00 Equity(157 [AEG])            2.946100         3.010   
                          Equity(680 [AXR])            4.492038         4.682   
                          Equity(753 [BBVA])           3.407867         3.540   
                          Equity(2035 [CX])            2.721500         2.820   
                          Equity(2036 [CXE])           4.588184         4.600   

                                              Percent Difference  \
2020-07-06 00:00:00+00:00 Equity(157 [AEG])             0.000407   
                          Equity(680 [AXR])             0.009069   
                          Equity(753 [BBVA])            0.002416   
                          Equity(2035 [CX])             0.042072   
                          Equity(2036 [CXE])            0.010421   

                                             Percent Filter  
2020-07-06 00:00:00+00:00 Equity(157 [AEG])            True  
                          Equity(680 [AXR])            True  
                          Equity(753 [BBVA])           True  
                          Equity(2035 [CX])            True  
                          Equity(2036 [CXE])           True

In [17]:
results.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 109 entries, (2020-07-06 00:00:00+00:00, Equity(157 [AEG])) to (2020-07-06 00:00:00+00:00, Equity(53294 [LINX]))
Data columns (total 4 columns):
30 Day Mean Close     109 non-null float64
Latest Close          109 non-null float64
Percent Difference    109 non-null float64
Percent Filter        109 non-null bool
dtypes: bool(1), float64(3)
memory usage: 3.5+ KB


### Pipeline Example

In [18]:
from quantopian.pipeline.filters import Q1500US

In [19]:
universe = Q1500US()

In [20]:
from quantopian.pipeline.data import morningstar

In [22]:
sector = morningstar.asset_classification.morningstar_sector_code.latest

In [23]:
energy_sector = sector.eq(309)

In [25]:
from quantopian.pipeline.factors import SimpleMovingAverage,AverageDollarVolume

In [26]:
dollar_volume = AverageDollarVolume(window_length=30)

In [27]:
dollar_volume.top(5)

NumExprFilter(expr='x_0 <= (5.0)', bindings={'x_0': Rank(AverageDollarVolume(...), method='ordinal')})

In [28]:
high_dollar_volume = dollar_volume.percentile_between(90,100)

In [29]:
top_open_prices = USEquityPricing.open.latest.top(50,mask=high_dollar_volume)

In [30]:
high_close_price = USEquityPricing.close.latest.percentile_between(90,100,mask=top_open_prices)

In [33]:
def make_pipeline():
    
    # Universe Q1500US
    base_universe = Q1500US()
    
    # Energy Sector
    energy_sector = sector.eq(309)
    
    # Make Mask of 1500US and Energy
    base_energy = base_universe & energy_sector
    
    # Dollar Volume (30 Days) Grab the Info
    dollar_volume = AverageDollarVolume(window_length=30)
    
    # Grab the top 5% in avg dollar volume
    high_dollar_volume = dollar_volume.percentile_between(95,100)
    
    # Combine the filters
    top_five_base_energy = base_energy & high_dollar_volume
    
    # 10 Day mean close
    mean_10 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=10,mask=top_five_base_energy)
    
    # 30 Day mean close
    mean_30 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=30,mask=top_five_base_energy)
    
    # Percent Difference
    percent_difference = (mean_10-mean_30)/mean_30
    
    # List of Shorts
    shorts = percent_difference < 0
    
    # List of Longs
    longs = percent_difference > 0
    
    # Final Mask/Filter for anything in shorts or longs
    securities_to_trade = (shorts | longs)
    
    # Return Pipeline
    return Pipeline(columns={
        'longs':longs,
        'shorts':shorts,
        'perc_diff':percent_difference
    },screen=securities_to_trade)

In [34]:
result = run_pipeline(make_pipeline(),'2020-07-29','2020-07-29')
result

longs  perc_diff shorts
2020-07-29 00:00:00+00:00 Equity(448 [APA])     True   0.027305  False
                          Equity(2564 [EOG])   False  -0.012486   True
                          Equity(3443 [HAL])    True   0.073536  False
                          Equity(5035 [MRO])   False  -0.028775   True
                          Equity(5213 [NBL])    True   0.072949  False
                          Equity(5634 [OKE])   False  -0.061510   True
                          Equity(5729 [OXY])   False  -0.053515   True
                          Equity(6928 [SLB])    True   0.021415  False
                          Equity(7990 [VLO])   False  -0.010547   True
                          Equity(8214 [WMB])    True   0.017902  False
                          Equity(8347 [XOM])   False  -0.012649   True
                          Equity(23112 [CVX])   True   0.006239  False
                          Equity(23998 [COP])  False  -0.016528   True
                          Equity(40852 [KMI])  False  -0.014793   True
                          Equity(41636 [MPC])   True   0.029647  False
                          Equity(42788 [PSX])  False  -0.040553   True